In [ ]:
import param
import panel as pn
import holoviews as hv
import pyvista as pv
from pyvista import examples
from scipy.ndimage import zoom
from functools import partial

css = '''
.custom-wbox > div.bk {
    padding-right: 10px;
}
.scrollable {
    overflow: auto !important;
}
'''
pn.extension('vtk', raw_css=[css])
hv.extension('bokeh')
hv.renderer('bokeh').theme = 'dark_minimal'
hv.opts.defaults(hv.opts.Image(invert_axes=True, responsive=True))

In [ ]:
class ImageSmoother(param.Parameterized):
    
    smooth_fun = param.Parameter(default=None)
    smooth_level = param.Integer(default=5, bounds=(1,10))
    order = param.Selector(default=1, objects=[1,2,3])
    
    def __init__(self, **params):
        super().__init__(**params)
        self._update_fun()
    
    @param.depends('order', 'smooth_level', watch=True)
    def _update_fun(self):
        self.smooth_fun = lambda x: zoom(x, zoom=self.smooth_level, order=self.order)
    
    @property
    def stream(self):
        return hv.streams.Params(self, parameters=['smooth_fun'])

In [ ]:
# Download datasets
head = examples.download_head()
brain = examples.download_brain()

In [ ]:
dataset_selection = pn.widgets.Select(value=head, options={'Head': head, 'Brain': brain})
volume = pn.panel(dataset_selection.value, sizing_mode='stretch_both', height=400, 
                  display_slices=True, orientation_widget=True,
                  render_background="#222222", colormap='blue2cyan')
dataset_selection.link(target=volume, value='object')

volume_controls = volume.controls(jslink=False, parameters=['render_background', 'display_volume', 'display_slices',
                                                            'slice_i', 'slice_j', 'slice_k', 'rescale'])
toggle_parallel_proj = pn.widgets.Toggle(name='Parallel Projection', value=False)
def update_camera_projection(*evts):
    volume.camera['parallelProjection'] = evts[0].new
    volume.param.trigger('camera')
toggle_parallel_proj.param.watch(update_camera_projection, ['value'], onlychanged=True)


stream_i = hv.streams.Params(volume, parameters=['slice_i'], rename={'slice_i':'si'})
stream_j = hv.streams.Params(volume, parameters=['slice_j'], rename={'slice_j':'sj'})
stream_k = hv.streams.Params(volume, parameters=['slice_k'], rename={'slice_k':'sk'})
sream_vol = hv.streams.Params(volume, parameters=['object'], rename={'object':'vol'})
stream_mapper = hv.streams.Params(volume, parameters=['mapper'])
smoother = ImageSmoother()
stream_smooth = smoother.stream

def hook_reset_range(plot, elem, x_range, y_range):
    bkplot = plot.handles['plot']
    bkplot.x_range.reset_start, bkplot.x_range.reset_end = x_range
    bkplot.y_range.reset_start, bkplot.y_range.reset_end = y_range

def image_slice_i(si, mapper, smooth_fun, vol):
    arr = vol.active_scalar.reshape(vol.dimensions, order='F')
    low = mapper['low'] if mapper else arr.min()
    high = mapper['high'] if mapper else arr.max()
    cmap = mapper['palette'] if mapper else 'fire'
    lbrt = vol.extent[4], vol.extent[2], vol.extent[5], vol.extent[3]
    im_i = hv.Image(smooth_fun(arr[si,::-1,:]), bounds=lbrt, kdims=['z','y'], vdims='Intensity')
    reset_fun = partial(hook_reset_range, x_range=(lbrt[1], lbrt[3]), y_range=(lbrt[0], lbrt[2]))
    return im_i.opts(hv.opts.Image(clim=(low, high), cmap=cmap, hooks=[reset_fun]))

def image_slice_j(sj, mapper, smooth_fun, vol):
    arr = vol.active_scalar.reshape(vol.dimensions, order='F')
    low = mapper['low'] if mapper else arr.min()
    high = mapper['high'] if mapper else arr.max()
    cmap = mapper['palette'] if mapper else 'fire'
    lbrt = vol.extent[4], vol.extent[0], vol.extent[5], vol.extent[1]
    im_j = hv.Image(smooth_fun(arr[::-1,sj,:]), bounds=lbrt, kdims=['z','x'], vdims='Intensity')
    reset_fun = partial(hook_reset_range, x_range=(lbrt[1], lbrt[3]), y_range=(lbrt[0], lbrt[2]))
    return im_j.opts(hv.opts.Image(clim=(low, high), cmap=cmap, hooks=[reset_fun]))

def image_slice_k(sk, mapper, smooth_fun, vol):
    arr = vol.active_scalar.reshape(vol.dimensions, order='F')
    low = mapper['low'] if mapper else arr.min()
    high = mapper['high'] if mapper else arr.max()
    cmap = mapper['palette'] if mapper else 'fire'
    lbrt = vol.extent[2], vol.extent[0], vol.extent[3], vol.extent[1]
    im_k = hv.Image(smooth_fun(arr[::-1,:,sk]), bounds=lbrt, kdims=['y','x'], vdims='Intensity')
    im_k.redim.range(y=(lbrt[0],lbrt[2]), x=(lbrt[1],lbrt[3]))
    reset_fun = partial(hook_reset_range, x_range=(lbrt[1], lbrt[3]), y_range=(lbrt[0], lbrt[2]))
    return im_k.opts(hv.opts.Image(clim=(low, high), cmap=cmap, hooks=[reset_fun]))

dmap_i = hv.DynamicMap(image_slice_i, streams=[stream_i, stream_mapper, stream_smooth, sream_vol])
dmap_j = hv.DynamicMap(image_slice_j, streams=[stream_j, stream_mapper, stream_smooth, sream_vol])
dmap_k = hv.DynamicMap(image_slice_k, streams=[stream_k, stream_mapper, stream_smooth, sream_vol])


controller = pn.WidgetBox(dataset_selection,
                          toggle_parallel_proj, *volume_controls[1:],
                          pn.Param(smoother, parameters=['smooth_level', 'order']),
                          pn.Spacer(height_policy='max'), sizing_mode='stretch_both', css_classes=['widget-box', 'custom-wbox'])

In [ ]:
template = """
{% extends base %}

<!-- goes in body -->
{% block postamble %}
<!-- <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css"> -->
<script type="text/javascript" src="http://code.jquery.com/jquery-1.11.1.min.js"></script>
<script type="text/javascript" src="https://golden-layout.com/files/latest/js/goldenlayout.min.js"></script>
<link type="text/css" rel="stylesheet" href="https://golden-layout.com/files/latest/css/goldenlayout-base.css" />
<link type="text/css" rel="stylesheet" href="https://golden-layout.com/files/latest/css/goldenlayout-dark-theme.css" />
{% endblock %}

<!-- goes in body -->
{% block contents %}
<div id='notebook_container'></div>
<script>
var config = {
    settings: {
        hasHeaders: true,
        constrainDragToContainer: true,
        reorderEnabled: true,
        selectionEnabled: false,
        popoutWholeStack: false,
        blockedPopoutsThrowError: true,
        closePopoutsOnUnload: true,
        showPopoutIcon: false,
        showMaximiseIcon: true,
        showCloseIcon: false
    },
    content: [{
        type: 'row',
        content:[
            {
                type: 'component',
                componentName: 'view',
                componentState: { model: '{{ embed(roots.controller) }}',
                                  title: 'Controls',
                                  width: 350,
                                  css_classes:['scrollable']},
                isClosable: false,
            },
            {
                type: 'column',
                content: [
                    {
                        type: 'row',
                        content:[
                            {
                                type: 'component',
                                componentName: 'view',
                                componentState: { model: '{{ embed(roots.scene3d) }}', title: '3D View'},
                                isClosable: false,
                            },
                            {
                                type: 'component',
                                componentName: 'view',
                                componentState: { model: '{{ embed(roots.slice_i) }}', title: 'Slice I'},
                                isClosable: false,
                            }
                        ]
                    },
                    {
                        type: 'row',
                        content:[
                            {
                                type: 'component',
                                componentName: 'view',
                                componentState: { model: '{{ embed(roots.slice_j) }}', title: 'Slice J'},
                                isClosable: false,
                            },
                            {
                                type: 'component',
                                componentName: 'view',
                                componentState: { model: '{{ embed(roots.slice_k) }}', title: 'Slice K'},
                                isClosable: false,
                            }
                        ]
                    }
                ]
            }
        ]
    }]
};
var title = $(document).find("title").text();
if (title=='VTKSlicer')
    var myLayout = new GoldenLayout( config );
else{
    var myLayout = new GoldenLayout( config, '#notebook_container' );
    $('#notebook_container').css({width: '100%', height:600})
}

myLayout.registerComponent('view', function( container, componentState ){
    const {width, css_classes} = componentState
    if(width)
        container.on('open', () => container.setSize(width, container.height))
    if (css_classes)
        css_classes.map((item) => container.getElement().addClass(item))
    container.setTitle(componentState.title)
    container.getElement().html(componentState.model);
    container.on('resize', () => window.dispatchEvent(new Event('resize')))
});

myLayout.init();
</script>
{% endblock %}
"""
tmpl = pn.Template(template)
controller.sizing_mode='stretch_both'
tmpl.add_panel('controller', pn.panel(controller, sizing_mode='stretch_both'))
tmpl.add_panel('scene3d', volume)
tmpl.add_panel('slice_i', pn.panel(dmap_i, sizing_mode='stretch_both'))
tmpl.add_panel('slice_j', pn.panel(dmap_j, sizing_mode='stretch_both'))
tmpl.add_panel('slice_k', pn.panel(dmap_k, sizing_mode='stretch_both'))


tmpl.servable(title='VTKSlicer')